Today we'll cover:

1. [Optimization and root finding](#Optimization-and-root-finding)
2. [Special functions](#Special-functions)
3. [Statistical functions](#Statistical-functions)

# Optimization and root finding

In [1]:
import numpy as np

In [2]:
from numpy import linalg as LA

In [3]:
from scipy.optimize import minimize, rosen

In [4]:
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

Let us run a simple [derivative free optimization](http://dx.doi.org/10.1137/1.9780898718768) method, called the [Nelder-Mead method](http://en.wikipedia.org/wiki/Nelder%E2%80%93Mead_method), starting from the above point `x0`. The function we will minimize is a popular test case in the optimization literature: the [Rosenbrock function](http://en.wikipedia.org/wiki/Rosenbrock_function).

In [5]:
res = minimize(rosen, x0, method='nelder-mead',
               options={'xtol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 339
         Function evaluations: 571


In [6]:
print res.x  # print the minimizer found

[ 1.  1.  1.  1.  1.]


In [7]:
print res  # res is an object with many fields

  status: 0
    nfev: 571
 success: True
     fun: 4.8611534334221152e-17
       x: array([ 1.,  1.,  1.,  1.,  1.])
 message: 'Optimization terminated successfully.'
     nit: 339


Let us try another deriviative free method called [Powell's Method](http://dx.doi.org/10.1093/comjnl/7.2.155). We will again use the Rosenbrock function for testing.

In [8]:
res = minimize(rosen, x0, method='powell',
               options={'xtol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 19
         Function evaluations: 1622


In [9]:
print res.x

[ 1.  1.  1.  1.  1.]


In [10]:
print res  # the structure of the returned objects depends on which method was used

  status: 0
 success: True
   direc: array([[  1.59217495e-06,   1.43301633e-06,   4.65187663e-06,
          9.58520273e-06,   2.09783685e-05],
       [ -5.72229228e-04,  -8.96827101e-04,  -2.05165897e-03,
         -4.12621496e-03,  -7.47022249e-03],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [ -6.33752967e-03,  -3.93431060e-03,  -1.22576639e-03,
          4.95313151e-05,   1.18066443e-04],
       [  3.39029104e-08,   7.58841519e-08,   1.38160426e-07,
          2.56876108e-07,   5.19973682e-07]])
    nfev: 1622
     fun: 2.0864733021357032e-21
       x: array([ 1.,  1.,  1.,  1.,  1.])
 message: 'Optimization terminated successfully.'
     nit: 19


We have so far use deriviative free or zero-order optimization methods. These methods only require function evaluations but can be slow to converge. Let us now look at first-order optimization methods, i.e., methods that evaluate derivatives of the function being optimized.

In [11]:
from scipy.optimize import rosen_der  # import the derivative of the Rosenbrock function

In [12]:
direction = np.random.randn(len(x0)); direction = direction/LA.norm(direction)  # choose a random unit vector

In [13]:
print direction, " has norm ", LA.norm(direction)

[-0.70649627 -0.00421516  0.32070061 -0.15388695  0.61181303]  has norm  1.0


In [14]:
eps = 1e-5  # a small number

In [15]:
rosen_change = rosen(x0 + eps * direction) - rosen(x0)  # actual change in the function in a given direction

In [16]:
approx_change = rosen_der(x0).dot(eps * direction)  # approx change using first order expansion

In [17]:
print "actual change = ", rosen_change, "\nchange using 1st order Taylor expansion =", approx_change

actual change =  -0.0108828003179 
change using 1st order Taylor expansion = -0.0108828622333


Let us now minimize the Rosenbrock function using the [BFGS method](http://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm). Note how we supply the derivative using the `jac` (for Jacobian) keyword argument.

In [18]:
res = minimize(rosen, x0, method='BFGS', jac=rosen_der,
               options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 52
         Function evaluations: 64
         Gradient evaluations: 64


In [19]:
print res.x

[ 1.  1.  1.  1.  1.]


In [20]:
print res  # note that hess_inv is an approximation to the Hessian computed internally by BFGS

   status: 0
  success: True
     njev: 64
     nfev: 64
 hess_inv: array([[ 0.00742784,  0.01239067,  0.02361262,  0.04673885,  0.09348604],
       [ 0.01239067,  0.02483023,  0.04731168,  0.09365448,  0.18731921],
       [ 0.02361262,  0.04731168,  0.09490941,  0.18787576,  0.37577275],
       [ 0.04673885,  0.09365448,  0.18787576,  0.3768795 ,  0.75378359],
       [ 0.09348604,  0.18731921,  0.37577275,  0.75378359,  1.51262571]])
      fun: 1.0115586212338213e-18
        x: array([ 1.,  1.,  1.,  1.,  1.])
  message: 'Optimization terminated successfully.'
      jac: array([  4.35961711e-10,   3.31877303e-09,  -3.38990125e-08,
         3.45087985e-08,  -9.65725278e-09])


Another first order method provided by `minimize` is the (Polak–Ribière) [Conjugate Gradient (CG) method](http://en.wikipedia.org/wiki/Nonlinear_conjugate_gradient_method).

In [21]:
res = minimize(rosen, x0, method='CG', jac=rosen_der,
               options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 174
         Function evaluations: 310
         Gradient evaluations: 310


In [22]:
print res.x

[ 0.99999985  0.99999971  0.99999943  0.99999888  0.99999774]


Now we will use a second-order optimization method, i.e. a method that uses Hessian evaluations. We will first import the Hessian of the Rosenbrock function from the `optimize` subpackage.

In [23]:
from scipy.optimize import rosen_hess

Let us check if the Hessian works as expected in a second-order Taylor expansion.

In [24]:
approx_change2 = rosen_der(x0).dot(eps * direction) + 0.5*(eps * direction).dot(rosen_hess(x0)).dot(eps * direction)

In [25]:
print "actual change = ", rosen_change, "\nchange using 2nd order Taylor approx. =", approx_change2

actual change =  -0.0108828003179 
change using 2nd order Taylor approx. = -0.0108828003175


We will now use the line search Newton-CG method (also called [truncated Newton method](http://www.neos-guide.org/content/truncated-newton-methods)).

In [26]:
res = minimize(rosen, x0, method='Newton-CG',
               jac=rosen_der, hess=rosen_hess,  # now we supply both Jacobian as well as Hessian
               options={'xtol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 24
         Function evaluations: 33
         Gradient evaluations: 56
         Hessian evaluations: 24


In [27]:
print res.x

[ 1.          1.          1.          0.99999999  0.99999999]


Let us end this section by using one of the root finding functions in `scipy.optimize`.

We will compute the [Omega constant](http://en.wikipedia.org/wiki/Omega_constant), namely the unique real number $\Omega$ that satisifes the equality $\Omega e^\Omega = 1$.

In [28]:
import math

In [29]:
def self_times_exp(x):
    return x*math.exp(x)

In [30]:
from scipy.optimize import bisect

In [31]:
Omega = bisect(lambda x: self_times_exp(x) - 1, 0, 1)  # find the root of x*exp(x) - 1 using bisection on the interval [0, 1]

In [32]:
print Omega

0.56714329041


In [33]:
print Omega * math.exp(Omega)

1.0


Another useful function is `scipy.optimize.golden` that implements the [Golden Section Search method](http://en.wikipedia.org/wiki/Golden_section_search) due to the statistician [Jack Kiefer](http://en.wikipedia.org/wiki/Jack_Kiefer_%28statistician%29). It searches for the minimum of a univariate function on an interval where there is a unique minimum. Each iteration reduces the interval to search in by a factor of $1/\phi$ where $\phi = \frac{1+\sqrt{5}}{2}$ is the [golden ratio](http://en.wikipedia.org/wiki/Golden_ratio).

In [34]:
from scipy.optimize import golden

Let us now find the unique minimum of the function $f(x) = xe^x$ on the interval $[-2, 0]$. Since $f'(x) = (1+x)e^x$, we know that the minimum is achived at $x=-1$ but let us find that numerically.

In [35]:
print golden(lambda x: x*math.exp(x), brack=(-2, 0))

-1.00000000199


# Special functions

`scipy.special` provides many special mathematical functions. Almost all of them are universal functions and so can be applied to ndarrays elementwise. There are far too many of them for us to cover them in any sufficient detail (it is best to consult the [reference page](http://docs.scipy.org/doc/scipy/reference/special.html#module-scipy.special) to get a good idea of what's available).

Here we simply find the Omega constant again via the [Lambert W function](http://en.wikipedia.org/wiki/Lambert_W_function) $W(z)$. It (or, to be more precise, its principal branch) is defined for $z \ge -1/e$ as the number $W(z) \ge -1$ such that $z = W(z) e^{W(z)}$. So, the Omega constant is nothing but $W(1)$.

In [36]:
from scipy.special import lambertw

In [37]:
print lambertw(1)  # returns a complex number

(0.56714329041+0j)


In [38]:
np.allclose(Omega, lambertw(1))  # check if our value, computed using bisect, agrees with the above

True

# Statistical functions

In [39]:
from scipy import stats

The `stats` subpackage provides many discrete rvs (random variables) and continuous rvs. Discrete rvs are instances of the class [`scipy.stats.rv_discrete`](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_discrete.html#scipy.stats.rv_discrete). Continuous rvs are instances of the class [`scipy.stats.rv_continuous`](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_discrete.html#scipy.stats.rv_discrete).

In [40]:
cont_rvs = [d for d in dir(stats) if isinstance(getattr(stats, d), stats.rv_continuous)]

In [41]:
disc_rvs = [d for d in dir(stats) if isinstance(getattr(stats, d), stats.rv_discrete)]

In [42]:
print "There are %d continuous and %d discrete distributions available." % (len(cont_rvs), len(disc_rvs))

There are 86 continuous and 13 discrete distributions available.


In [43]:
from scipy.stats import expon  # import the exponential distribution

In [44]:
samples = expon.rvs(size=50); print samples  # draw 50 samples from the distribution

[  6.16755471e-01   2.00155581e+00   1.59410481e+00   1.81967825e+00
   2.70458588e-03   1.58646217e+00   5.88185633e-01   1.17508374e+00
   1.20904118e+00   1.04800396e+00   8.46532636e-01   1.02705972e+00
   1.91742834e+00   4.22041585e-01   1.77882825e+00   7.62140037e-01
   1.36567755e-02   1.08967911e-01   1.99686270e+00   2.97167298e-01
   1.71192926e+00   4.87026792e+00   4.03509102e-01   7.37263365e-02
   5.21680628e-02   5.11189818e+00   5.74790641e+00   5.15406037e-01
   2.26265629e+00   8.76046179e-01   2.77714060e-01   6.98332771e-02
   4.34312074e-01   3.38551665e+00   2.37087137e+00   9.53821031e-01
   1.91474175e+00   3.98271574e-01   5.90163490e-01   1.87667420e+00
   5.00362067e-02   2.13805686e-01   1.89013920e-01   1.44922118e+00
   1.25632013e-01   1.82912169e+00   3.27947748e-01   3.79692763e-01
   1.54928454e-01   2.06582918e-01]


In [45]:
print np.mean(samples), expon.mean()  # check closeness of empirical mean with true mean

1.19271353482 1.0


Let's print the standard deviation, variance and entropy of an exponential rv.

In [46]:
print "SD = %f, Var = %f, Entropy = %f" % (expon.std(), expon.var(), expon.entropy())

SD = 1.000000, Var = 1.000000, Entropy = 1.000000


In [47]:
print expon.pdf(0)  # p.d.f. at 0

1.0


In [48]:
print expon.cdf(0)  # c.d.f. at 0

0.0


In [49]:
print expon.ppf(.75)  # inverse cdf, useful to get percentiles (75th in this case)

1.38629436112


In [50]:
quantiles = np.arange(.1, 1, .1); print quantiles
    

[ 0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9]


In [51]:
np.allclose(expon.cdf(expon.ppf(quantiles)), quantiles)  # check if ppf is indeed inverse cdf

True

In [52]:
expon.expect(lambda x: x**2)  # expectation of the function x^2 under the expon p.d.f.

2.0

We know that $\int x^k e^{-x} dx = k!$. Let's test if `expect` works as expected.

In [53]:
from scipy.special import factorial

In [54]:
np.allclose(factorial(np.arange(10)), np.array([expon.expect(lambda x: x**k) for k in range(10)]))

True

Since expectations of $x^k$ are just the (uncentered) moments, we could have also equivalently done the following.

In [55]:
np.allclose(factorial(np.arange(10)), np.array([expon.moment(k) for k in range(10)]))

True

The `median()` method returns the median. Let us test it on 4 popular continuous distributions.

In [56]:
rv_names = ['expon', 'norm', 'cauchy', 'uniform']

In [57]:
rv_objects = [getattr(stats, rv_name) for rv_name in rv_names]

In [58]:
[np.allclose(rv.median(), rv.ppf(0.5)) for rv in rv_objects]  # median should be inverse cdf at 0.5

[True, True, True, True]

Now let us work with a discrete rv, say Poisson.

In [59]:
from scipy.stats import poisson

In [60]:
samples = poisson.rvs(mu=1, size=50)  # note how we have to supply the mu parameter in this case (no default assumed)

In [61]:
print samples

[0 1 2 0 1 1 3 1 2 1 1 0 1 0 2 1 1 0 0 1 0 2 1 0 1 0 3 2 0 2 1 0 2 1 2 3 3
 1 2 0 1 4 0 1 0 0 0 1 1 1]


In [62]:
print np.mean(samples), poisson(mu=1).mean()

1.08 1.0


If you don't like supplying the `mu` argument all the time, you can also create a "frozen rv" (see discussion [here](http://docs.scipy.org/doc/scipy/reference/tutorial/stats.html#freezing-a-distribution)).

In [63]:
poisson1 = poisson(mu=1)  # create frozen rv

In [64]:
p_mean, p_var, p_skew = poisson1.stats('mvs')  # get mean, variance and skewness

In [65]:
print "Mean = %f, Var = %f, Skewness = %f" % (p_mean, p_var, p_skew)

Mean = 1.000000, Var = 1.000000, Skewness = 1.000000


Poisson($\mu = 1$) has p.m.f. $p(i) = e^{-1} \frac{1}{i!}, i \in \{0, 1, 2, \ldots\}$.

In [66]:
known_pmf = np.exp(-1)*(1/factorial(np.arange(10)))  # note that factorial (imported from scipy.special) is a ufunc

In [67]:
np.allclose(known_pmf, poisson1.pmf(np.arange(10)))

True